## Homework 2

Instructions
Ваше завдання створити візуалізації, котрі відповідають на наступні питання:

- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

Ви самостійно маєте обрати спосіб візуалізації. Ви також маєте написати короткий супровідний текст до кожної візуалізації, котрий пояснює, чому ви обрали саме цей спосіб презентації даних, які ще альтернативи ви розглядали, та чому зупинились саме на цьому варіанті (які його переваги та недоліки).

Складові генерації: АЕС,ТЕЦ,ВДЕ,ТЕС,ГЕС,ГАЕС. Споживання не включає обсяги закачки ГАЕС. Міждержавні перетоки: Україна - Білорусь та РФ, Україна - ЄС, Україна - Молдова. Одиниця виміру - МВт.
Наразі не створено жодного вигляду для цього ресурсу.

In [61]:
import   altair as alt
import   pandas as pd
import   datetime
alt.renderers.enable('default')

RendererRegistry.enable('default')

## 1. Data investigation

In [62]:
df = pd.read_excel('./2014-2020.xlsx')
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61368 entries, 0 to 61367
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Час/Дата     61368 non-null  object 
 1   AES          61368 non-null  int64  
 2   TEC          61368 non-null  int64  
 3   VDE          61368 non-null  int64  
 4   TES          61368 non-null  int64  
 5   GES          61367 non-null  float64
 6   GAES_GEN     61368 non-null  int64  
 7   CONSUMPTION  61368 non-null  int64  
 8   GAES_PUMP    61367 non-null  float64
 9   UK_BLR_RUS   61367 non-null  float64
 10  UK_EURO      61367 non-null  float64
 11  UK_MLD       61367 non-null  float64
 12  Unnamed: 12  1 non-null      float64
dtypes: float64(6), int64(6), object(1)
memory usage: 6.1+ MB


## 2. Data preprocessing

In [64]:
# Delete strange column and ones we won't use

df.drop("Unnamed: 12",  axis='columns', inplace=True)
df.drop("GAES_PUMP",    axis='columns', inplace=True)
df.drop("UK_BLR_RUS",   axis='columns', inplace=True)
df.drop("UK_EURO",      axis='columns', inplace=True)
df.drop("UK_MLD",       axis='columns', inplace=True)

In [65]:
# Find rows where are Nan values

df[df.isnull().any(axis=1)]

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION
17544,24-31.12.2018,10533,2345,11,5882,NaN,0,17883


In [66]:
# Lets drop this row 17544, due to avoid incomplete picture

df.drop(df.index[[17544]], inplace=True)

In [67]:
# Transform columns

df["Hour"]    = df["Час/Дата"].str.split("-",expand=True)[0]
df["Date"]    = df["Час/Дата"].str.split("-",expand=True)[1]
df["Day"]     = df["Date"].str.split(".",expand=True)[0]
df["Month"]   = df["Date"].str.split(".",expand=True)[1]
df["Year"]    = df["Date"].str.split(".",expand=True)[2]
df["Hour"]    = pd.to_numeric(df["Hour"])
df["Day"]     = pd.to_numeric(df["Day"])
df["Month"]   = pd.to_numeric(df["Month"])
df["Year"]    = pd.to_numeric(df["Year"])
df["Date"]    = pd.to_datetime(df["Date"])
df['Weekday'] = pd.Series(df["Date"]).dt.day_name()

In [68]:
# Add season column for next visualization

season_month = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}


df["Season"] = df.apply (lambda row: season_month.get(row["Month"]), axis=1)

In [69]:
df.drop("Час/Дата", axis='columns', inplace=True)

In [70]:
df.head()

,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,Hour,Date,Day,Month,Year,Weekday,Season
0,9235,2039,621,4942,385.0,0,16693,24,2020-12-31,31,12,2020,Thursday,Winter
1,9221,2159,707,5549,470.0,0,17805,23,2020-12-31,31,12,2020,Thursday,Winter
2,9249,2377,709,5906,1000.0,0,18870,22,2020-12-31,31,12,2020,Thursday,Winter
3,9256,2499,702,6329,909.0,322,19887,21,2020-12-31,31,12,2020,Thursday,Winter
4,9213,2521,702,6640,823.0,602,20387,20,2020-12-31,31,12,2020,Thursday,Winter


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61367 entries, 0 to 61367
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   AES          61367 non-null  int64         
 1   TEC          61367 non-null  int64         
 2   VDE          61367 non-null  int64         
 3   TES          61367 non-null  int64         
 4   GES          61367 non-null  float64       
 5   GAES_GEN     61367 non-null  int64         
 6   CONSUMPTION  61367 non-null  int64         
 7   Hour         61367 non-null  int64         
 8   Date         61367 non-null  datetime64[ns]
 9   Day          61367 non-null  int64         
 10  Month        61367 non-null  int64         
 11  Year         61367 non-null  int64         
 12  Weekday      61367 non-null  object        
 13  Season       61367 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(10), object(2)
memory usage: 7.0+ MB


## 3. Plotting

### Task 1. Як змінювалась структура генерації електроенергії за роками?

### Task 2. Як залежить споживання електроенергії від дня року та години доби?

### Task 3. Як змінюється генерація електроенергії з різних джерел впродовж доби?

### Task 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

### Task 5. Як змінюється споживання електроенергії впродовж тижня?